In [26]:
file_path = 'day_5_input.txt'
with open(file_path, 'r') as file:
    seed_data = file.readline()
    file.seek(0)
    data = file.read()

seeds = [int(seed) for seed in seed_data.split(':')[1].strip().split()]

def parse(row):
    number_lists = row.split(':')[1].split('\n')
    return [list(map(int, item.split())) for item in number_lists if item]

maps = {row.split(':')[0]: parse(row) for row in data.split('\n\n')[1:]}
maps.keys()

dict_keys(['seed-to-soil map', 'soil-to-fertilizer map', 'fertilizer-to-water map', 'water-to-light map', 'light-to-temperature map', 'temperature-to-humidity map', 'humidity-to-location map'])

In [27]:
def process(n, map_rows):
    for dest_start, source_start, range_length in map_rows:
        if source_start <= n < source_start + range_length:
            return dest_start + (n - source_start)

    return n

locations = []

for seed in seeds:
    soil = process(seed, maps['seed-to-soil map'])
    fertilizer = process(soil, maps['soil-to-fertilizer map'])
    water = process(fertilizer, maps['fertilizer-to-water map'])
    light = process(water, maps['water-to-light map'])
    temp = process(light, maps['light-to-temperature map'])
    humidity = process(temp, maps['temperature-to-humidity map'])
    location = process(humidity, maps['humidity-to-location map'])
    locations.append(location)

min(locations)

84470622

In [154]:
test = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""

# data = test.strip()
# seed_data = data.splitlines()[0]

file_path = 'day_5_input.txt'
with open(file_path, 'r') as file:
    seed_data = file.readline()
    file.seek(0)
    data = file.read()


def generate_seeds(seed_data):
    seed_numbers = [int(seed) for seed in seed_data.split(':')[1].strip().split()]
    seeds = []
    for i in range(0, len(seed_numbers), 2):
        start = seed_numbers[i]
        end = start + seed_numbers[i + 1]
        seeds.append((start, end))
    return seeds

def parse(row):
    number_lists = row.split(':')[1].split('\n')
    return [list(map(int, item.split())) for item in number_lists if item]

def process_range(input_range, map_rows, output=None, step=None):
    if output is None:
        output = []
        
    input_start, input_end = input_range
    for dest_start, source_start, range_length in map_rows:
        source_end = source_start + range_length
        
        # entire range in mapping -> map the entire range
        if source_start <= input_start < source_end and input_end < source_end:
            output_start = dest_start + (input_start - source_start)
            output_end = dest_start + (input_end - source_start)
            # print('mapping entire range', (input_start, input_end), (output_start, output_end), (source_start, dest_start), range_length)
            output.append((output_start, output_end))
            return output

        # start in mapping, end not:
        elif source_start <= input_start < source_end and input_end >= source_end:
            # print('start in mapping, end not', (input_start, source_end - 1), (source_end, input_end), source_start, source_end)
            process_range((input_start, source_end - 1), map_rows, output)
            process_range((source_end, input_end), map_rows, output)
            break

        # end in mapping, start not
        elif source_start <= input_end < source_end and input_start < source_start:
            # print('end in mapping, start not', (input_start, source_start - 1), (source_start, input_end), source_start, source_end)
            process_range((input_start, source_start - 1), map_rows, output)
            process_range((source_start, input_end), map_rows, output)
            break

    # Range not in any mapping => keep it as is
    else:
        # print('doing nothing', input_start, input_end)
        output.append((input_start, input_end))    
    return output

seeds = generate_seeds(seed_data)
maps = {row.split(':')[0]: parse(row) for row in data.split('\n\n')[1:]}

def run(input_ranges, map_rows):
    output_ranges = []
    for input_range in input_ranges:
        result = process_range(input_range, map_rows)
        output_ranges.extend(result)

    return output_ranges

soil = run(seeds, maps['seed-to-soil map'])
fertilizer = run(soil, maps['soil-to-fertilizer map'])
water = run(fertilizer, maps['fertilizer-to-water map'])
light = run(water, maps['water-to-light map'])
temp = run(light, maps['light-to-temperature map'])
humidity = run(temp, maps['temperature-to-humidity map'])
location = run(humidity, maps['humidity-to-location map'])

min(location_range[0] for location_range in location)

26714516

In [138]:
result = process_range((57, 70), [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]], output=None)
print(result)


# (57, 70) => (53, 56) (61, 70)

[(53, 56), (61, 70), (57, 70)]


In [ ]:
# seed  soil
# 0     0
# 1     1
# ...   ...
# 48    48
# 49    49
# 50    52
# 51    53
# 52    54
# 53    55
# ...   ...
# 96    98
# 97    99
# 98    50
# 99    51

# (48, 53) => (48, 49), (52, 55)